### Kütüphanelerin Yüklenmesi

In [1]:
import pandas as pd

### Veri Yüklenmesi

In [2]:
data = pd.read_csv("data/clean_data.csv")

In [3]:
data.sample(5)

,Unnamed: 0,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,...,2nd_Road_Number,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Year
354530,354681,200650F42B858,293750.0,73140.0,-3.501074,50.548271,50,3,1,1,...,0,None within 50 metres,No physical crossing within 50 meters,Darkness: Street lights present and lit,Raining without high winds,Wet/Damp,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,1,2006
212077,212207,200601QK50088,521160.0,185860.0,-0.253622,51.558566,1,3,2,1,...,406,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,1,2006
149100,149176,2.01E+12,619480.0,136560.0,1.132570,51.086281,46,3,1,1,...,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Wet/Damp,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,1,2005
247135,247278,2.01E+12,424190.0,560690.0,-1.623934,54.940314,10,3,2,1,...,326,None within 50 metres,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Wet/Damp,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,1,2006
174282,174362,2005555B03362,407960.0,93530.0,-1.888573,50.741260,55,3,2,1,...,0,None within 50 metres,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,No Special Conditions or Not Specified,No Carriageway Hazards or Not Specified,1,2005


### Okunan veri setinden gerekli olan sütunların filtrelenmesi

In [4]:
data_filter = data[["Local_Authority_(Highway)", "Latitude", "Longitude", "Year"]]

In [15]:
len(data_filter["Local_Authority_(Highway)"].unique())

207

In [6]:
data_filter["Year"].unique()

array([2005, 2006, 2007, 2009, 2010, 2011, 2012, 2013, 2014], dtype=int64)

In [7]:
data_filter.to_csv("data_filter.csv")

In [17]:
import geopandas
from geodatasets import get_path

In [18]:
gdf_uk = geopandas.read_file("data/uk_lad.geojson")

In [19]:
gdf_uk_locals = gdf_uk[["name","reference"]]

gdf_uk_locals.sample(5)

,name,reference
28,Peterborough,E06000031
304,Kingston upon Thames,E09000021
276,Walsall,E08000030
138,Tunbridge Wells,E07000116
103,Uttlesford,E07000077


In [28]:
gdf_uk_locals = gdf_uk_locals.rename(columns={"reference": "Local_Authority_(Highway)"})

In [29]:
gdf_uk_locals.sample(5)

,name,Local_Authority_(Highway)
37,Windsor and Maidenhead,E06000040
317,Buckinghamshire,E06000060
303,Kensington and Chelsea,E09000020
76,High Peak,E07000037
116,Havant,E07000090


In [30]:
data_filter1 = data_filter

In [33]:
len(data_filter1)

1469609

In [31]:
import pandas as pd

# Merge the two datasets using the 'Postal Code' column
merged_data = data_filter1.merge(gdf_uk_locals, on='Local_Authority_(Highway)', how='left')

# 'how' parameter specifies how to perform the merge. 'left' means we keep all rows from the first dataset.

# The resulting merged_data DataFrame will contain district names added to the first dataset based on the 'Postal Code' column.

# You can save the merged dataset to a new CSV file if needed:
merged_data.to_csv('merged_dataset.csv', index=False)

In [32]:
merged_data.sample(5)

,Local_Authority_(Highway),Latitude,Longitude,Year,name
310637,E06000055,52.137988,-0.444248,2006,Bedford
1191450,E09000031,51.574903,-0.019634,2013,Waltham Forest
50588,E06000050,53.174699,-2.656915,2005,Cheshire West and Chester
901111,E09000004,51.452366,0.151810,2011,Bexley
625788,E08000032,53.787187,-1.740874,2009,Bradford


### Filtrelenen verinin yıllara göre ayrılması

In [8]:
data_2005 = data_filter[data_filter["Year"] == 2005]
data_2006 = data_filter[data_filter["Year"] == 2006]
data_2007 = data_filter[data_filter["Year"] == 2007]
data_2009 = data_filter[data_filter["Year"] == 2009]
data_2010 = data_filter[data_filter["Year"] == 2010]
data_2011 = data_filter[data_filter["Year"] == 2011]
data_2012 = data_filter[data_filter["Year"] == 2012]
data_2013 = data_filter[data_filter["Year"] == 2013]
data_2014 = data_filter[data_filter["Year"] == 2014]

In [9]:
data_2005.to_csv("data_2005.csv")
data_2006.to_csv("data_2006.csv")
data_2007.to_csv("data_2007.csv")
data_2009.to_csv("data_2009.csv")
data_2010.to_csv("data_2010.csv")
data_2011.to_csv("data_2011.csv")
data_2012.to_csv("data_2012.csv")
data_2013.to_csv("data_2013.csv")
data_2014.to_csv("data_2014.csv")

### TABLEAU LINK